## Validation Site Maps and Wind Roses

Caleb Phillips (caleb.phillips@nrel.gov)

A small notebook to make maps and windroses for the Bergey (and eventually OneEnergy) sites.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

import fiona
import site_index
from dw_tap.data_processing import filter_obstacles

import plotly.express as px

%matplotlib inline
%config InlineBackend.figure_format='retina'

obstacle_data_dir = "01 Bergey Turbine Data/3dbuildings_geojson"

In [ ]:
index = site_index.SiteIndex()

### 100m Radius Obstacle Maps

In [ ]:
all_obstacles_100m = []
for tid in index.tids():
    index_row = index.lookup_by_tid(tid)
    z_turbine = index_row["Hub Height (m)"]
    lat = index_row["Latitude"]
    lon = index_row["Longitude"]
    
    obstacle_data_dir = "01 Bergey Turbine Data/3dbuildings_geojson"
    obstacle_data_file = "%s/%sv2.json" % (obstacle_data_dir, tid)
    
    if os.path.exists(obstacle_data_file):
        obstacle_df = filter_obstacles(tid,gpd.read_file(obstacle_data_file), 
                                       include_trees=True, 
                                       turbine_height_for_checking=z_turbine,
                                       limit_to_radius_in_m=100.0,
                                       turbine_lat_lon=(lat,lon))
        obstacle_df["tid"] = tid
        
        print("%s" % (tid,))
        
        colors = {'tree': "#b2df8a", 'building': "#1f78b4"} # colors from colorbrewer.org
        ax = obstacle_df.plot(color=obstacle_df["feature_type"].map(lambda x: colors[x]))
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        #ax.set_title(tid)
        ax.set_xlim(lon-0.0015,lon+0.0015)
        ax.set_ylim(lat-0.001, lat+0.001)
        plt.plot(lon, lat, marker="*", markersize=20, markeredgecolor="red", markerfacecolor="red")
        plt.show()
        
        all_obstacles_100m.append(obstacle_df)
    else:
        print("Can't access: %s. Skipping" % obstacle_data_file)

all_obstacles_100m = pd.concat(all_obstacles_100m)

### 500m Radius Obstacle Maps (all obstacles)

In [ ]:
all_obstacles = []
for tid in index.tids():
    index_row = index.lookup_by_tid(tid)
    z_turbine = index_row["Hub Height (m)"]
    lat = index_row["Latitude"]
    lon = index_row["Longitude"]
    
    obstacle_data_dir = "01 Bergey Turbine Data/3dbuildings_geojson"
    obstacle_data_file = "%s/%sv2.json" % (obstacle_data_dir, tid)
    
    if os.path.exists(obstacle_data_file):
        obstacle_df = filter_obstacles(tid,gpd.read_file(obstacle_data_file), 
                                       include_trees=True, 
                                       turbine_height_for_checking=z_turbine)
        obstacle_df["tid"] = tid
        
        print("%s" % (tid,))
        
        colors = {'tree': "#b2df8a", 'building': "#1f78b4"} # colors from colorbrewer.org
        ax = obstacle_df.plot(color=obstacle_df["feature_type"].map(lambda x: colors[x]))
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        #ax.set_title(tid)
        ax.set_xlim(lon-0.0015*5,lon+0.0015*5)
        ax.set_ylim(lat-0.001*5, lat+0.001*5)
        plt.plot(lon, lat, marker="*", markersize=20, markeredgecolor="red", markerfacecolor="red")
        plt.show()
        
        all_obstacles.append(obstacle_df)
    else:
        print("Can't access: %s. Skipping" % obstacle_data_file)

all_obstacles = pd.concat(all_obstacles)

### Wind Roses

In [ ]:
wtk = pd.read_csv("01 Bergey Turbine Data/wtk.csv.bz2")
wtk["ws_rounded"] = wtk["ws"].round()
wtk["wd_rounded"] = 10*(wtk["wd"]/10).astype(int)
dc = wtk.copy()[["tid","wd_rounded","ws_rounded"]]
dc["n"] = 1
dc = dc.groupby(["tid","wd_rounded","ws_rounded"]).sum()
dc = dc.reset_index()
dc['p'] = np.round(100*dc['n']/61368,1)

In [ ]:
for tid in index.tids():
    fig = px.bar_polar(dc[dc["tid"] == tid], r="p", theta="wd_rounded",
                   color="ws_rounded",
                   color_continuous_scale= 'Jet',
                   labels={"ws_rounded": "Windspeed (m/s)","p": "Percent (%)"},
                   title=tid)
    fig.show()